In [85]:
import requests
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import logging

data = pd.DataFrame({
    'URL': [],
    'tel': [],
    'mail': []
})


In [86]:

re.findall(p, '+7 (343) 289-02-59')

[]

In [87]:
URL = 'https://c-ns.ru'

PHONE_PATTERNS = [
    r'\(\d{4}\) \d{3}-\d{3}',
    r'\(\d{4}\) \d{2}-\d{2}-\d{2}',
    
    # 1 3 3 (2 2 | 4)
    r'\(?\+\d[\s-]?\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2}',

    # 1|0 4 2 (2 2 | 4)
    r'\+\d?[\s-]?\(?\d{4}\)?[\s-]\d{2}[\s-]\d{2}[\s-]?\d{2}',
    
    # 8
    r'8[\s-]\(?\d{3}\)?[\s-]\d{3}[\s-]\d{2}[\s-]?\d{2}',
    
    # 3 2 2
    r'\(?\d{3}\)?[\s-]\d{2}[\s-]?\d{2}',
    
    # 3 1 3
    r'\(?\d{3}\)?[\s-]\d{1}[\s-]?\d{3}',
    
    # 3 3 1 3
    r'\+\d[\s-]?\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{1}[\s-]?\d{3}',
]

for p in PHONE_PATTERNS:
    print(re.findall(p, '+7 (343) 289-02-59'))

[]
[]
['+7 (343) 289-02-59']
[]
[]
['289-02-59']
[]
[]


In [89]:
def extract_mail(line):
    match = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', line)
    return match.group(0)

def get_phone_numbers(line):
    global PHONE_PATTERNS
    for pattern in PHONE_PATTERNS:
        if len(re.findall(pattern, line)) > 0:
            return tuple(np.unique(re.findall(pattern, line)))
        

def get_mails(text):
    if '@' in text:
        try:
            return extract_mail(text)
        except:
            pass
        

If information on the page

In [90]:
urls = pd.read_csv('ready_domains.csv', header=None)
urls = urls[urls.columns[0]]
urls[:5]

0                  https://1470umto.ru
1                    https://23city.ru
2                 https://abc-sklad.ru
3                 https://absrealty.ru
4    https://aeroflotovec-2.landorg.ru
Name: 0, dtype: object

In [91]:

def parse_one_url(url):
    req = requests.get(url)
    soup = bs(req.text, 'html.parser')
    texts = soup.find_all(text=True)
    texts = u" ".join(t.strip() for t in texts)
    l = [url,  get_phone_numbers(texts), get_mails(texts)];
    
    return l

In [92]:
parse_one_url(URL)

['https://c-ns.ru', ('+7 812 610 00 55',), 'info@c-ns.ru']

In [94]:
from multiprocessing import Pool

# This thing is pretty fast
with Pool(processes=32) as mp_pool:
    result = mp_pool.map(parse_one_url, urls)
    
result

In [ ]:
df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)
df


/tmp/ipykernel_7961/1435031242.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(result, columns =['url', 'tel', 'mail'], dtype = float)


,url,tel,mail
0,https://1470umto.ru,"((3022) 217-719,)",info@1470umto.ru
1,https://23city.ru,"(8 (800) 775-18-10,)",info@23city.ru
2,https://abc-sklad.ru,"(+7 999 555 00 55, +7999 555 00 55, +79995550055)",None
3,https://absrealty.ru,"(+7 495 156-42-73, +74950237526, +74951042537,...",u002F96b8b7066d2f4fc49867fc7b8ab981ee@sentry.i...
4,https://aeroflotovec-2.landorg.ru,"(+79625884609,)",snp.a-2@mail.ru
...,...,...,...
240,https://zemexx.ru,"(+7 495 989-10-70,)",zer@zemexx.ru
241,https://zemlo.ru,"(+7 812 603 43 73, +7 921 996 53 63, +7 921 99...",info@zemlo.ru
242,https://zen.yandex.ru/polis_group,None,None
243,https://zhk-premier-sochi.ru,"(+7 (862) 225-72-30,)",None


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     245 non-null    object
 1   tel     217 non-null    object
 2   mail    122 non-null    object
dtypes: object(3)
memory usage: 5.9+ KB


In [ ]:
df[df.tel.isna()]

,url,tel,mail
6,https://akademicheskiy.org,None,akademicheskiy@kortros.ru
7,https://akdomrb.ru,None,None
10,https://an-nks.ru,None,None
37,https://berezka.org,None,None
45,https://bsfc.com/novostroiki/246-yugge/?utm_ca...,None,bsfc@bsfc.com
62,https://dmitrovka-zn.ru,None,None
80,https://fond214pk.ru/objects/%D0%B3-%D0%BF%D0%...,None,None
86,https://gk-osnova.ru,None,None
99,https://insta-mukhanovasvetlana.clients.site,None,None
100,https://interstroi.com.ru,None,None
